# Dataframe for Dynamic Pricing

## Objectives

This document intends to briefly describe one alternative for the ETL process for the Dynamic Pricing problem at Olist.  
As the end product, we would like to have a table with the proper structure for forecasting techniques.  
The datasets will be described along the way.

### Importing packages

In [ ]:
import pandas as pd
import datetime

# Experimentation: Loops

### 1 - Read dataframes

In [72]:
# ORDERS
orders = pd.read_csv('data/orders.csv', dtype={'product_gtin':'str','origin_zip':'str','destination_zip':'str'})
orders['purchase_timestamp'] = pd.to_datetime(orders['purchase_timestamp'])

# PRODUCT INFO
prod_info = pd.read_csv('data/product_info.csv', dtype={'gtin':'str'})
prod_info['created_at'] = pd.to_datetime(prod_info['created_at'])

# PRODUCT TYPE
product_type = pd.read_csv('data/product_type.csv', dtype={'value':str})

# PRODUCT HISTORY
history = pd.read_csv('data/full_history.csv', dtype={'gtin':str})
history['updated_at'] = pd.to_datetime(history['updated_at'])

# COMPETITION
competition = pd.read_csv('data/competition.csv', dtype={'gtin':'str'})
competition['updated_at'] = pd.to_datetime(competition['updated_at'])

# TODO PRODUCTS WITHOUT SALES

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 2 - CREATE BASIC ETL

In [ ]:
#orders.product_gtin.value_counts()

In [71]:
gtin = '7898164712364'

# Get creation date for selected gtin
created_at = prod_info[prod_info['gtin']==gtin]['created_at'].min().date()

name = prod_info[prod_info['gtin']==gtin]['name'].iloc[0]
description = prod_info[prod_info['gtin']==gtin]['description'].iloc[0]


# Create full range df from creation date to today
freq = 'D'
end_date = '2020-04-05'
t_df = pd.DataFrame(data=pd.date_range(start=created_at, end=end_date, freq=freq), columns=['date'])


## ORDERS DATASET
# Filter orders dataset
t_orders = orders[orders['product_gtin']==gtin][['purchase_timestamp','product_gtin', 'price','freight_value', 'commission_product_olist','commission_freight_olist']].copy()

# ADD ARTIFICIAL COLUMN WITH THE NUMBER OF ORDERS -> WILL BE USED TO COUNT THE ORDERS BY HOUR
t_orders['orders'] = 1


# GROUPS ORDERS BY CHOSEN FREQUENCY
t_orders = t_orders.groupby([pd.Grouper(key='purchase_timestamp', freq=freq),'product_gtin'])\
                  .agg({'price':'mean', 
                        'freight_value':'mean',
                        'commission_product_olist':'mean',
                        'commission_freight_olist':'mean',
                        'orders':'sum'}).reset_index()

# MERGE
t_df = pd.merge(t_df, t_orders, how='left', left_on='date', right_on='purchase_timestamp')
t_df.drop('purchase_timestamp', axis=1, inplace=True)
t_df['product_gtin'] = gtin


# ## COMPETITION DATASET
# # FILTER GTIN
t_competition = competition[competition['gtin'] == gtin].copy()
# Exclude 0 values
t_competition = t_competition[t_competition['value']>0]
# filter competition on extremely low and high prices
std = t_competition['value'].std()
mean = t_competition['value'].mean()
t_competition = t_competition[(t_competition['value']>= mean-3*std) & (t_competition['value']<= mean+3*std)]

# Group by freq and get minimum value
t_competition = t_competition.groupby([pd.Grouper(key='updated_at', freq=freq),'gtin']).min()[['value']].reset_index()

# Merge datasets
t_df = pd.merge(t_df, t_competition,
         left_on=['date','product_gtin'], right_on=['updated_at', 'gtin'],
         how='left')

t_df.rename(columns={'value':'competition_price'}, inplace=True)

t_df = t_df[['date', 'product_gtin', 'competition_price', 'price', 'freight_value',
       'commission_product_olist', 'commission_freight_olist', 'orders']].copy()

t_df['competition_price'].fillna(method='ffill', inplace=True)
t_df['competition_price'].fillna(method='bfill', inplace=True)


# # PRODUCT TYPE DATASET
t_product_type = product_type[product_type['value']==gtin].copy()

t_df = pd.merge(left=t_df, right=t_product_type,
         left_on='product_gtin', right_on='value', how='left')[['date', 'product_gtin', 'competition_price',
                                                    'price', 'freight_value', 'commission_product_olist', 
                                                    'commission_freight_olist', 'orders','product_type', 
                                                    'product_type_group']]




# # HISTORY DATASET
t_history = history[history['part_number']==gtin].copy()

t_history.head()

# # filter zero and outlier values
# t_history = t_history[t_history['offer'] > 0].copy()
# std = t_history['offer'].std()
# mean = t_history['offer'].mean()

# t_history = t_history[(t_history['offer']>= mean-3*std) & (t_history['offer']<= mean+3*std)]

# t_history = t_history.groupby([pd.Grouper(key='updated_at', freq='D'), 'gtin'])\
#                          .agg({'stock':['max', 'min', 'mean'], 
#                                'price':['max', 'min', 'mean'],
#                                'offer':['max', 'min', 'mean']}).reset_index()

# t_history.columns = t_history.columns.droplevel(0)

# t_history.columns = ['date', 'product_gtin',
#                    'stock_max', 'stock_min', 'stock_avg',
#                    'price_max','price_min','price_avg',
#                    'offer_max','offer_min','offer_avg']

# t_df = pd.merge(left=t_df, right=t_history,
#                   left_on=['date', 'product_gtin'], right_on=['date', 'product_gtin'],
#                   how='left')

# # forward fill agg measures then backfill
# t_df[['stock_max', 'stock_min',
#        'stock_avg', 'price_max', 'price_min', 'price_avg', 'offer_max',
#        'offer_min', 'offer_avg']] = t_df[['stock_max', 'stock_min',
#                                             'stock_avg', 'price_max', 'price_min', 'price_avg', 'offer_max',
#                                             'offer_min', 'offer_avg']]\
#                                             .fillna(method='pad')\
#                                             .fillna(method='bfill')

# t_df['name'] = name
# t_df['description'] = description



/usr/local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,gtin,part_number,canonical_sku,seller_product_sku,channel_slug,external_id,price,offer,stock,updated_at


In [73]:
def etl_function(gtin, orders, prod_info, product_type, competition, history):
    # Get creation date for selected gtin
    created_at = prod_info[prod_info['gtin']==gtin]['created_at'].min().date()

    name = prod_info[prod_info['gtin']==gtin]['name'].iloc[0]
    description = prod_info[prod_info['gtin']==gtin]['description'].iloc[0]


    # Create full range df from creation date to today
    freq = 'D'
    end_date = '2020-04-05'
    t_df = pd.DataFrame(data=pd.date_range(start=created_at, end=end_date, freq=freq), columns=['date'])
    t_df['product_gtin'] = gtin

    ## ORDERS DATASET
    # Filter orders dataset
    t_orders = orders[orders['product_gtin']==gtin][['purchase_timestamp','product_gtin', 'price','freight_value', 'commission_product_olist','commission_freight_olist']].copy()

    # ADD ARTIFICIAL COLUMN WITH THE NUMBER OF ORDERS -> WILL BE USED TO COUNT THE ORDERS BY HOUR
    t_orders['orders'] = 1


    # GROUPS ORDERS BY CHOSEN FREQUENCY
    t_orders = t_orders.groupby([pd.Grouper(key='purchase_timestamp', freq=freq),'product_gtin'])\
                      .agg({'price':'mean', 
                            'freight_value':'mean',
                            'commission_product_olist':'mean',
                            'commission_freight_olist':'mean',
                            'orders':'sum'}).reset_index()

    # MERGE
    t_df = pd.merge(t_df, t_orders, how='left', left_on='date', right_on='purchase_timestamp')
    t_df.drop('purchase_timestamp', axis=1, inplace=True)
    
    t_df['product_gtin'] = gtin

    ## COMPETITION DATASET
    # FILTER GTIN
    t_competition = competition[competition['gtin'] == gtin].copy()
    # Exclude 0 values
    t_competition = t_competition[t_competition['value']>0]
    # filter competition on extremely low and high prices
    std = t_competition['value'].std()
    mean = t_competition['value'].mean()
    t_competition = t_competition[(t_competition['value']>= mean-3*std) & (t_competition['value']<= mean+3*std)]

    # Group by freq and get minimum value
    t_competition = t_competition.groupby([pd.Grouper(key='updated_at', freq=freq),'gtin']).min()[['value']].reset_index()

    # Merge datasets
    t_df = pd.merge(t_df, t_competition,
             left_on=['date','product_gtin'], right_on=['updated_at', 'gtin'],
             how='left')

    t_df.rename(columns={'value':'competition_price'}, inplace=True)

    t_df = t_df[['date', 'product_gtin', 'competition_price', 'price', 'freight_value',
           'commission_product_olist', 'commission_freight_olist', 'orders']].copy()

    t_df['competition_price'].fillna(method='ffill', inplace=True)
    t_df['competition_price'].fillna(method='bfill', inplace=True)




    # PRODUCT TYPE DATASET
    t_product_type = product_type[product_type['value']==gtin].copy()

    t_df = pd.merge(left=t_df, right=t_product_type,
             left_on='product_gtin', right_on='value', how='left')[['date', 'product_gtin', 'competition_price',
                                                        'price', 'freight_value', 'commission_product_olist', 
                                                        'commission_freight_olist', 'orders','product_type', 
                                                        'product_type_group']]

    # HISTORY DATASET
    t_history = history[history['gtin']==gtin].copy()

    # filter zero and outlier values
    t_history = t_history[t_history['offer'] > 0].copy()
    std = t_history['offer'].std()
    mean = t_history['offer'].mean()

    t_history = t_history[(t_history['offer']>= mean-3*std) & (t_history['offer']<= mean+3*std)]

    t_history = t_history.groupby([pd.Grouper(key='updated_at', freq='D'), 'gtin'])\
                             .agg({'stock':['max', 'min', 'mean'], 
                                   'price':['max', 'min', 'mean'],
                                   'offer':['max', 'min', 'mean']}).reset_index()

    t_history.columns = t_history.columns.droplevel(0)

    t_history.columns = ['date', 'product_gtin',
                       'stock_max', 'stock_min', 'stock_avg',
                       'price_max','price_min','price_avg',
                       'offer_max','offer_min','offer_avg']

    t_df = pd.merge(left=t_df, right=t_history,
                      left_on=['date', 'product_gtin'], right_on=['date', 'product_gtin'],
                      how='left')

    # forward fill agg measures then backfill
    t_df[['stock_max', 'stock_min',
           'stock_avg', 'price_max', 'price_min', 'price_avg', 'offer_max',
           'offer_min', 'offer_avg']] = t_df[['stock_max', 'stock_min',
                                                'stock_avg', 'price_max', 'price_min', 'price_avg', 'offer_max',
                                                'offer_min', 'offer_avg']]\
                                                .fillna(method='pad')\
                                                .fillna(method='bfill')

    t_df['name'] = name
    t_df['description'] = description
    
    t_df['orders'].fillna(value=0,inplace=True)
    t_df.loc[t_df["price"].isnull(),'price'] = t_df["offer_avg"]
    
    return t_df

In [74]:
len(gtins)

698

In [76]:
prods = pd.read_csv('data/products.csv', dtype={'gtin':'str'})
dfs = []
failed = []
gtins = prods.gtin.unique().tolist()

for gtin in gtins:
    try:
        t_df = etl_function(gtin, orders, prod_info, product_type, competition, history)
        dfs.append(t_df)
    except:
        print(gtin)
        failed.append(gtin)

40232931314
639853916499
606034863491
70896842121
732094204209
606529194192
695977684652
793574682894
20714170486
85000010587
751320753359
603922842064
606529089313
631384908165
606529884864
200000097790
650728044215
735810185448
751320234186
10450015189
27242868854
669818570800
40232926778
7426844271162
720355985363
40094936601
735810138666
669818571548
78000804690
76950450370
92756008973
669818568524
748323522587
602168928082
45496590017
603786426219
656879891258
656871234121
695937550119
769493100382
2960000075355
85854241793
639853121220
82184046791
34264495326
889698364676
742186937359
653437599090
86800097679
73228106052
49694508997
2960000359806
671072038248
700083859563


In [77]:
len(dfs)

644

In [78]:
len(failed)

54

In [79]:
df = pd.concat(dfs)

In [82]:
df[df['product_gtin']=='7890201094952']

,date,product_gtin,competition_price,price,freight_value,commission_product_olist,commission_freight_olist,orders,product_type,product_type_group,stock_max,stock_min,stock_avg,price_max,price_min,price_avg,offer_max,offer_min,offer_avg,name,description
0,2019-08-07,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
1,2019-08-08,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
2,2019-08-09,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
3,2019-08-10,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
4,2019-08-11,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
5,2019-08-12,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
6,2019-08-13,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
7,2019-08-14,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,1000.0,5.0,154.250000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
8,2019-08-15,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,5.0,5.0,5.000000,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...
9,2019-08-16,7890201094952,119.00,119.00,NaN,NaN,NaN,0.0,NaN,NaN,1000.0,5.0,336.666667,119.00,119.00,119.00,119.00,119.00,119.00,Kit Berço Americano Trança Menino 11 Peças Nuv...,TOTAL 11 PEÇAS\n\n1 Cabeceira-----------------...


In [83]:
df.to_csv('data/timeseries644.csv', index=False)

### OLD APPROACH

### Dataframe with complete range

Let's start by creating a dataframe with every data point we want to measure. It will contain hourly intervals, equally spaced between two dates.  
You can change the parameters in the cell below:

In [ ]:
freq = 'D' # 'H' is for hourly data. You can use 'D' for days, 'W' for last day of week, 'Q' for quarter, etc.

start_date = '1/1/2017' # MM/DD/YYYY
end_date = '4/5/2020' # MM/DD/YYYY

In [ ]:
df = pd.DataFrame(data=pd.date_range(start=start_date, end=end_date, freq=freq),
                  columns=['date'])

df.head(2)

In [ ]:
df.tail(2)

### Orders dataset

Let's investigate our first dataset "orders.csv". It contains information for every sale that was executed in a given period of time.  
Our main idea is to aggregate sales data and merge it with our dates dataframe (df).

In [ ]:
# READING ORDERS DATA AND TRANSFORMING COLUMNS
orders = pd.read_csv('data/orders.csv', dtype={'product_gtin':'str','origin_zip':'str',
                                               'destination_zip':'str'})

orders['purchase_timestamp'] = pd.to_datetime(orders['purchase_timestamp'])

In [ ]:
orders.head()

In [ ]:
orders.product_gtin.nunique()

###### Selecting specific product

In the orders dataset, we have sales information for 647 different products.  
For simplification purposes, let's choose one single product for analysis and manipulation of the dataset.  
To do so, choose a single gtin (str) in the cell below.

- gtin = '7908243600137'
- product_sku = 'PRD41RHHBF9NQR74'
- canonical_sku = 'PRDIM5XOLWMUM0HU'


In [ ]:
filter_orders = orders[['purchase_timestamp','product_gtin',
                        'price','freight_value',
                        'commission_product_olist','commission_freight_olist']].copy()

# ADD ARTIFICIAL COLUMN WITH THE NUMBER OF ORDERS -> WILL BE USED TO COUNT THE ORDERS BY HOUR
filter_orders['orders'] = 1


# GROUPS ORDERS BY CHOSEN FREQUENCY
filter_orders = filter_orders.groupby([pd.Grouper(key='purchase_timestamp', freq=freq),'product_gtin'])\
                  .agg({'price':'mean', 
                        'freight_value':'mean',
                        'commission_product_olist':'mean',
                        'commission_freight_olist':'mean',
                        'orders':'sum'}).reset_index()

In [ ]:
filter_orders.head()

##### merge date_range dataframe and orders

Now that we have the number of sales of a given product by hour, along with its average price, average shipping values and average commissions (percentage), we can join with our date range dataframe.  

The 'NaN' rows are the ones that did not have any sales.

In [ ]:
# MERGE DF and Filter orders

merged = pd.merge(df, filter_orders, how='left', left_on='date', right_on='purchase_timestamp')
merged.drop('purchase_timestamp', axis=1, inplace=True)

In [ ]:
merged.head()

In [ ]:
# Checking some rows with sales

merged[merged['orders'].notnull()].head()

### Product_info dataset

Another available dataset is the "product_info.csv". We will use it to check when a product was created in our database to fine tune the starting point of our time series.

In [ ]:
prod_info = pd.read_csv('data/product_info.csv', dtype={'gtin':'str'})

prod_info['created_at'] = pd.to_datetime(prod_info['created_at'])

In [ ]:
filter_prod = prod_info[['created_at','gtin','name']]

In [ ]:
filter_prod = filter_prod.groupby(['gtin','name']).min().reset_index()

In [ ]:
def truncate_date(x):
    return datetime.date(x.year,x.month,x.day)

In [ ]:
filter_prod['created_date'] = filter_prod['created_at'].map(truncate_date)

In [ ]:
filter_prod.head()

##### Filter merged based on creation date

In [ ]:
merged = pd.merge(merged, filter_prod, left_on='product_gtin', right_on='gtin', how='left')

In [ ]:
merged['created_date'] = pd.to_datetime(merged['created_date'])

In [ ]:
# Keep dates only where the product existed.
#merged = merged[merged['date']>=merged['created_date']].copy()

In [ ]:
merged.head()

### Competition

Now, let's get our competitor's prices by the same frequency.

In [ ]:
# Read data and format it
competition = pd.read_csv('data/competition.csv', dtype={'gtin':'str'})

competition['updated_at'] = pd.to_datetime(competition['updated_at'])

In [ ]:
competition.head()

In [ ]:
# filter out zero values
competition = competition[competition['value'] > 0]

In [ ]:
gr = competition.groupby('gtin').agg({'value':['mean', 'std']}).reset_index()

In [ ]:
gr.head()

In [ ]:
comp_m = pd.merge(competition, gr, left_on='gtin', right_on='gtin')

In [ ]:
comp_m.columns = ['updated_at', 'gtin', 'value', 'slug', 'mean','std']

In [ ]:
comp_m['upper'] = comp_m['mean'].values + 3*comp_m['std'].values
comp_m['lower'] = comp_m['mean'].values - 3*comp_m['std'].values

In [ ]:
comp_m.head()

In [ ]:
comp_m.shape

In [ ]:
# filter comp_m on extremely low and high prices
comp_m = comp_m[(comp_m['value']>= comp_m['lower']) & (comp_m['value']<= comp_m['upper'])]

In [ ]:
comp_m.shape

In [ ]:
# Group by freq and get minimum value
comp_m = comp_m.groupby([pd.Grouper(key='updated_at', freq=freq),'gtin']).min()[['value']].reset_index()

In [ ]:
comp_m.head()

##### merge competition prices with merged dataframe

In [ ]:
#merged['day'] = merged['date'].apply(lambda x: x.date())

In [ ]:
merged = pd.merge(merged, comp_m,
         left_on=['date','product_gtin'], right_on=['updated_at', 'gtin'],
         how='left')

merged.rename(columns={'value':'competition_price'}, inplace=True)

merged = merged[['date', 'product_gtin',  'competition_price', 'price', 'freight_value',
       'commission_product_olist', 'commission_freight_olist', 'orders']].copy()

In [ ]:
merged.head()

In [ ]:
merged[merged['orders']==0]

In [ ]:
merged.sort_values(by=['product_gtin', 'date'])

We will forward fill the competition price.

In [ ]:
merged['competition_price'].fillna(method='ffill', inplace=True)

In [ ]:
merged.head()

In [ ]:
merged.tail()

### Product type and group

Let's gather some information about the type of the analyzed product.

This particular gtin belongs to the product_type "Kit para cabelo" (Hair Care Kit).  
The product belongs to the broader group "beleza_saude" (beauty_health)

In [ ]:
product_type = pd.read_csv('data/product_type.csv', dtype={'value':str})

product_type.head()

product_type[product_type['value']==gtin]

merged.columns

merged = pd.merge(left=merged, right=product_type,
         left_on='product_gtin', right_on='value')[['date', 'product_gtin', 'competition_price',
                                                    'price', 'freight_value', 'commission_product_olist', 
                                                    'commission_freight_olist', 'orders','product_type', 
                                                    'product_type_group']]

In [ ]:
merged.head()

### Olist history of prices
### TODO

For this particular case, we will only read historical data for the selected gtin. This will be changed once olist can share data on other products.  
Due to internal issues, the price history could not be gathered.


In [ ]:
# history = pd.read_csv('data/product_history.csv', dtype={'gtin':str})
# history['updated_at'] = pd.to_datetime(history['updated_at'])

### SOME CHARTS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
merged.columns

In [ ]:
trace0 = go.Scatter(x=merged.date,
                   y=merged.orders, mode='lines')

trace1 = go.Scatter(x=merged.date,
                    y=merged.competition_price, mode='markers')

trace2 = go.Scatter(x=merged.date,
                    y=merged.price,
                    mode='markers')

trace3 = go.Scatter(x=merged.date,
                    y=merged.freight_value,
                    mode='lines')


iplot([trace0, trace1, trace2, trace3])

In [ ]:
plt.figure(figsize=(12,8))

plt.style.use('fivethirtyeight')

sns.lineplot(x='date', y='orders', data=merged)
sns.scatterplot(x='date', y='competition_price', data=merged, label='competitor')
sns.scatterplot(x='date', y='price', data=merged, label='olist')

plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
sum(merged['orders'])